# Figure 1: Map color legend to phospho results for MAPK Signaling diagram

In [1]:
import pandas as pd

import cptac
import cptac.utils as u
import plot_utils as p

In [2]:
print('cptac version:', cptac.version())

cptac version: 0.8.6


In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
'''
This function returns the color used in Figure 1 for every 
differential expression/p-value status of a certain phosphosite.
@Param site: String. Name of phosphosite column.
@Param df: df with pval and differential expressions. NaN dropped.
@Param cancer: String. Cancer name used in df columns. 
'''

def check_status_color(site, df, cancer):
    df = df.loc[df['Phospho'] == site]
    pval = float(df[cancer+'_P_Value'])
    dif_expression = float(df[cancer+'_Median'])
    color = ''
    if pval >= 0.1:
        color += 'grey'
    else:
        if pval < .05:
            color += 'dark_'
        if dif_expression < 0:
            color += 'blue'
        elif dif_expression > 0:
            color += 'red'
        
    return color
    

# Step 1: Get tables with MAPK signaling phosphoproteomics.

In [5]:
df = pd.read_csv("../Make_Tables/csv/phospho_MAPK.csv")
luad_df = pd.read_csv("../Make_Tables/csv/Single_Cancer/LUAD_phospho_MAPK.csv")
e_df = pd.read_csv("../Make_Tables/csv/Single_Cancer/EC_phospho_MAPK.csv")
c_df = pd.read_csv("../Make_Tables/csv/Single_Cancer/CO_phospho_MAPK.csv")

FileNotFoundError: [Errno 2] No such file or directory: '../Make_Tables/csv/phospho_MAPK.csv'

# Step 2: Create df with all phosphosites mapped with colors


Color Legend:
* red - differential expression > 0
* blue - differential expression < 0
* grey - not significant
* white - no data / not enough data for t-test

All sites used in Figure 1 have at least 1 FDR corrected p-value < 0.1. The darker hue of a color represents a  p-value < 0.05 (dark_red, dark_blue). 

In [ ]:
e_df_2 = e_df.dropna() # drop nan (p-vals nan when not enough data to do t-test)
e_df_2['color'] = e_df_2['Phospho'].apply(lambda x: check_status_color(x, e_df_2, 'EC'))
#e_df_2

In [ ]:
luad_df_2 = luad_df.dropna()
luad_df_2['color'] = luad_df_2['Phospho'].apply(lambda x: check_status_color(x, luad_df_2, 'LUAD'))
#luad_df_2

# Step 3: Show data for sites used in KRAS Figure 

* Shown phosphosites are significant in at least 1 cancer.

In [ ]:
# Get list of phosphosites sig in at least 1 cancer
e = e_df.loc[e_df['EC_P_Value'] < 0.1]
e_sig_phospho = list(e.Phospho)

luad = luad_df.loc[luad_df['LUAD_P_Value'] < 0.1]
luad_sig_phospho = list(luad.Phospho)

fig_phospho = list(set(luad_sig_phospho + e_sig_phospho))

In [ ]:
combined = e_df_2.merge(luad_df_2, on = 'Phospho', how = 'outer', suffixes = ['_Endo', '_Luad'])
fig_data = combined[combined['Phospho'].isin(fig_phospho)]
fig_data.color_Luad = fig_data.color_Luad.fillna('white')
fig_data.color_Endo = fig_data.color_Endo.fillna('white')
fig_data

# Get number of significant phosphosites

The manuscript mentions the number of phosphosites significant in each cancer and in multiple cancers.

In [ ]:
print('LUAD')
luad_sig_2 = luad_df.loc[luad_df['LUAD_P_Value'] < 0.1]
print('p-val < 0.1 : ', len(luad_sig_2))

print('\nEC')
e_sig_2 = e_df.loc[e_df['EC_P_Value'] < 0.1]
print('p-val < 0.1 : ', len(e_sig_2))

print('\nCO')
c_sig_2 = c_df.loc[c_df['CO_P_Value'] < 0.1]
print('p-val < 0.1 : ', len(c_sig_2))

In [ ]:
print('Sig in both')
p = 0.1
sig = df.loc[df['EC_P_Value'] < p]
sig = sig.loc[sig['LUAD_P_Value'] < p]
print('p-val < ',p,' : ', len(sig), '\n')
print(list(sig.Phospho), '\n')

# Check that Colon has measurements in the MAPK pathway

In [ ]:
# Get protein column
e_sig_2['Protein'], e_sig_2['Site'] = e_sig_2.Phospho.str.split('_', 1).str
luad_sig_2['Protein'], luad_sig_2['Site'] = luad_sig_2.Phospho.str.split('_', 1).str
c_df['Protein'], c_df['Site'] = c_df.Phospho.str.split('_', 1).str

In [ ]:
e_list = list(e_sig_2.Protein)
luad_list = list(luad_sig_2.Protein)
prot = list(set(e_list + luad_list))
print('Proteins')
print('Num proteins with sig sites in EC and LUAD:', len(prot), 'proteins')

prot_sig = c_df[c_df.Protein.isin(prot)].dropna()
print(len(prot_sig.Protein.unique()), 'of those proteins found in CO')

In [ ]:
# does colon have data for sites sig in other cancers?
e_list = list(e_sig_2.Phospho)
luad_list = list(luad_sig_2.Phospho)
sites = list(set(e_list + luad_list))
print('Phosphosites')
print('Num sig sites in EC or LUAD:', len(sites) ,'sites')

sites_sig = c_df[c_df.Phospho.isin(sites)]
print(len(sites_sig.Phospho.unique()), 'sig sites found in CO')